# Statistical analysis of a time series dataset of US macroeconomic variables

This project is a part of the assessment for a model validation job at UBS Mumbai. Here I have done a statistical analysis of a linear regression model, supplied by UBS, on a time series dataset of US macroeconomic variables.

### Table of contents
* [Our data](#data)
* [Task 1: Replication of results](#task1)
* [Task 2: Outlier detections](#task2)
* [Task 3: Autocorrelation of residuals](#task3)
* [Task 4: Bootstrapping of standard errors](#task4)
* [Summary](#summary)

In [1]:
#Importing necessary python libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm


## Our data: <a class="anchor" id="data"></a>
Quarterly time series of US macroeconomic variables from 1950 until 2000.

In [2]:
#Importing the data file to pandas dataframe
df_data = pd.read_csv("~/Documents/assignment/data.csv",sep='|')
df_data.head()

,date,gdp,consumption,invest,government,dpi,cpi,m1,tbill,unemp,population,inflation,interest
0,1950 Q1,1610.5,1058.9,198.1,361.0,1186.1,70.6,110.20,1.12,6.4,149.461,NaN,NaN
1,1950 Q2,1658.8,1075.9,220.4,366.4,1178.1,71.4,111.75,1.17,5.6,150.260,4.5071,-3.3404
2,1950 Q3,1723.0,1131.0,239.7,359.6,1196.5,73.2,112.95,1.23,4.6,151.064,9.9590,-8.7290
3,1950 Q4,1753.9,1097.6,271.8,382.5,1210.0,74.9,113.93,1.35,4.2,151.871,9.1834,-7.8301
4,1951 Q1,1773.5,1122.8,242.9,421.9,1207.9,77.3,115.08,1.40,3.5,152.393,12.6160,-11.2160


In [3]:
#Checking for the null values in the dataset
df_data[df_data.isnull().any(axis=1)]

,date,gdp,consumption,invest,government,dpi,cpi,m1,tbill,unemp,population,inflation,interest
0,1950 Q1,1610.5,1058.9,198.1,361.0,1186.1,70.6,110.2,1.12,6.4,149.461,NaN,NaN


We find that only the first row (date = 1950 Q1) contains the NaN in the <b>inflation</b> and the <b>interest</b> columns. For now this is not a problem since we are not immediately concerned with these columns.

## Task 1: Replication of results <a class="anchor" id="task1"></a>

#### The linear regression model:
\begin{equation}
D(\text{consumption})_t = \alpha + \beta_1 D(\text{dpi})_t + \beta_2 D(\text{unemp})_t + \epsilon_t
\end{equation}

where $D()$ denotes the first difference and $\epsilon_t$ are the residuals.

We now construct a dataset with only the columns (consumption, dpi, unemp) relevant to the model in question.

In [3]:
df_reduced = df_data[['consumption','dpi','unemp']]
df_reduced.head()

,consumption,dpi,unemp
0,1058.9,1186.1,6.4
1,1075.9,1178.1,5.6
2,1131.0,1196.5,4.6
3,1097.6,1210.0,4.2
4,1122.8,1207.9,3.5


#### Constructing the dataset with first differences of the reduced dataset df_reduced

In [4]:
#Creating the dataset with first difference
df_reduced_diff = pd.Series.diff(df_reduced)
df_reduced_diff.columns = ['D(consumption)','D(dpi)','D(unemp)']

#Removing the first row since it will definitely be null
df_reduced_diff.dropna(inplace=True) #In this dataset only the first column has NA values.
df_reduced_diff.head()

,D(consumption),D(dpi),D(unemp)
1,17.0,-8.0,-0.8
2,55.1,18.4,-1.0
3,-33.4,13.5,-0.4
4,25.2,-2.1,-0.7
5,-31.4,17.9,-0.4


### We now construct a linear regression model and fit with the data to get the model parameters and errors

In [5]:
# Defining independent and dependent variables as X and y respectively
n = len(df_reduced_diff)
X = df_reduced_diff[['D(dpi)','D(unemp)']]
y = df_reduced_diff['D(consumption)']

# We create a table of independent variables with first column being "1",
# whose coefficient will give the intercept.
X_with_const = np.empty([n,3])
X_with_const[:, 0] = 1
X_with_const[:, 1:] = X.values

# Creating the regression model using statsmodel.api
# and fitting with the data X_with_const and y
reg_model = sm.OLS(y.values, X_with_const)
reg_result = reg_model.fit()
reg_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     50.31
Date:                Tue, 25 Oct 2022   Prob (F-statistic):           2.00e-18
Time:                        14:26:09   Log-Likelihood:                -898.12
No. Observations:                 203   AIC:                             1802.
Df Residuals:                     200   BIC:                             1812.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.2848      1.911      8.522      0.000      12.517      20.053
x1             0.3557      0.048      7.444      0.000       0.261       0.450
x2           -16.0149      3.792     -4.223      0.000     -23.493      -8.537
==============================================================================
Omnibus:                       12.022   Durbin-Watson:                   1.767
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               14.257
Skew:                           0.459   Prob(JB):                     0.000802
Kurtosis:                       3.919   Cond. No.                         110.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The p-values for the t-statics ($P>|t|$ column in the OLS Regression Results) are <b>0</b>, upto three places of decimal, for all the parameters $\alpha$, $\beta_1$ and $\beta_2$. This implies that the relationship between the variables of the regression model is <b>statistically significant</b>.

## Task 2: Outlier detections <a class="anchor" id="task2"></a>

We start with constructing the time series of residuals ($\epsilon_t$).

In [10]:
# Coefficients taken from the OLS Regression Results
alpha = 16.2848
beta1 = 0.3557
beta2 = -16.0149

# Computing residuals
residuals = (df_reduced_diff['D(consumption)']-alpha-beta1*df_reduced_diff['D(dpi)']
             -beta2*df_reduced_diff['D(unemp)'])

Now we find quariles $Q_{1/4}$ and $Q_{3/4}$, the interquartile range IQR, the range

\begin{equation}
\left[Q_{1/4}(x)-1.5\times\text{IQR},Q_{3/4}(x)+1.5\times\text{IQR}\right]
\end{equation}

and finally the number of outliers.

In [11]:
# Determining quartiles and IQR of the sorted residual array
Q1 = np.quantile(residuals,1/4)
Q3 = np.quantile(residuals,3/4)
IQR = Q3-Q1

# Range
range_min = Q1-1.5*IQR
range_max = Q3+1.5*IQR

# Outliers
outliers = [elem for elem in residuals if (elem < range_min or elem > range_max)]
number_of_outliers = len(outliers)
print("number of outliers =",number_of_outliers)

number of outliers = 12


In a sample size of 203 there are 12 outliers ($5.9\%$) which is significant. The presence of outliers implies that there is a possibility of a better model. We can therefore try looking for the dependence of $D$(consumption) on other variables in the dataset. We can see if by including them we can reduce the number of outliers. One can also consider the possibility of non-linear dependence of $D$(consumption) on $D$(dpi) or $D$(unemp).

###### Trying to explain the above point:
Suppose that there is another variable $D$(x) which determines $D$(consumption), i.e.,

\begin{equation}
D(\text{consumption})_t = \alpha + \beta_1 D(\text{dpi})_t + \beta_2 D(\text{unemp})_t + \beta_3 D(\text{x}) + \epsilon_t^{\text{new}}
\end{equation}

where $\epsilon_t = \epsilon_t^{\text{new}} + \beta_3 D(\text{x})$. If $|D(\text{x})|$ is of large for a data point, then it can make $|\epsilon_t|$ large and make it an outlier. (This is true only if $|D(\text{x})|$ is small for residuals which are not outliers.)

Note that there is a simplifying assumption in the above example that other coefficients remain unaffected by the introduction of the variable $|D(\text{x})|$, which may not be true.

## Task 3: Autocorrelation of residuals <a class="anchor" id="task3"></a>

We compute the Durbin-Watson (DW) statistic of the residuals defined as
\begin{equation}
DW = \frac{\sum_{t=2}^n\left(\epsilon_t-\epsilon_{t-1}\right)^2}{\sum_{t=1}^n\epsilon_t^2}
\end{equation}

In [12]:
# Constructing the dataset of the first difference of the residuals.
residuals_diff = pd.Series.diff(residuals)
residuals_diff.dropna(inplace=True) #Removing NaN which is only there in the first row.

dw_num = np.dot(residuals_diff,residuals_diff) #Numerator of the DW
dw_den = np.dot(residuals,residuals) #Denominator of the DW
dw = dw_num/dw_den
print("Durbin-Watson =",dw)

Durbin-Watson = 1.767157842189361


Note that the DW statistics is calculated by the statsmodel.api.OLS and it agrees with the result we got above (see OLS Regression Results).

$DW=0$ implies that all residues are equal, i.e., they are strongly positively autocorrelated. If they are not autocorrelated, i.e., $\sum_{t}\epsilon_t\epsilon_{t-1}=0$, we get $DW\approx2$. $DW>2$ implies negative autocorrelation.

A strong autocorrelation implies that residuals closer in time are predictable (similar for positive and opposite in sign for negative). They are not random and therefore need to be modeled. A strong autocorrelation therefore suggests a scope for improvement in regression model.

We have $DW=1.767$ which is close to $2$ and therefore our residuals appear to be sufficiently random.

We can further compute the $p$-value ($P>|DW-2|$) and check if it is greater than $0.05$ to confirm the above claim.

## Task 4: Bootstrapping of standard errors <a class="anchor" id="task4"></a>

In this task we generate 10000 samples of size $n$, which is the size of the original dataset, using random sampling with replacement.

In [13]:
bootst_param = 0
bootst_stderr = 0
sample_number = 10000
k=0
while k<sample_number:
    #Random sampling indices from 0 to (n-1), n = len(df_reduced_diff)
    sample_index = np.random.choice(n,n,replace=True)
    #Creating sample_X_with_const and sample_y using X_with_const and y defined in Task 1
    sample_X_with_const = X_with_const[sample_index]
    sample_y = y.iloc[sample_index]
    #Creating the regression model using statsmodel.api and fitting with the data X_with_const and y
    sample_reg_model = sm.OLS(sample_y.values, sample_X_with_const)
    sample_reg_result = sample_reg_model.fit()
    sample_param = sample_reg_result.params
    sample_stderr = sample_reg_result.bse
    bootst_param = bootst_param + sample_param
    bootst_stderr = bootst_stderr + sample_stderr
    k=k+1
    
bootst_param = bootst_param/sample_number
bootst_stderr = bootst_stderr/sample_number

print("bootstrapped alpha =",bootst_param[0])
print("bootstrapped beta1 =",bootst_param[1])
print("bootstrapped beta2 =",bootst_param[2])

print("bootstrapped standard error in alpha =",bootst_stderr[0])
print("bootstrapped standard error in beta1 =",bootst_stderr[1])
print("bootstrapped standard error in beta2 =",bootst_stderr[2])

bootstrapped alpha = 16.234066713916764
bootstrapped beta1 = 0.3571133619694947
bootstrapped beta2 = -16.024204576780605
bootstrapped standard error in alpha = 1.9078486685876852
bootstrapped standard error in beta1 = 0.04792992006196084
bootstrapped standard error in beta2 = 3.815217254638791


While bootstrapping we generate 10000 random samples from the original data and average over the standart error of the parameters obtained for individual samples. This process is expected to reduce the effect of outliers on the result. In our case we see that the standard errors obtained by bootstrapping is almost same as the default standard errors (see OLS Regression Results of Task 1), which implies that the standard error of the model parameters calculated by the statsmodel.api.OLS in Task 1 is not significantly affected by the outliers

## Summary <a class="anchor" id="summary"></a>

In this project we tested a linear regression model for the quarterly time series of US macroeconomic variables, relating the first difference in <b>consumptions</b> to that in <b>real disposable personal income (dpi)</b> and in <b>unemployment rate (unemp)</b>. We find the coefficients and intercept as follows

Coefficient of $D$(dpi) = $0.3557$, standard error = $0.048$

Coefficient of $D$(unemp) = $-16.0149$, standard error = $3.792$

Intercept = $16.2848$, standard error = $1.911$

The model suggests that our dataset contains 12 outliers in a sample size of 103, which indicates towards a possibility of improvement in the model. However the bootstrapped parameters and standard errors turn out to be almost equal to the default parameters and standard errors which suggests the outliers have no significant effect on the model.

We also checked for autocorrelation of residuals and found that the Durbin-Watson statistic is $1.767$ which is close to two and hence implies that the residuals are not autocorrelated and there is no need to model the residuals. I think we may however need to find the $p$-value for the DW statistic in order to confirm the claim.